In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.python.framework import ops
ops.reset_default_graph()

In [2]:
sentences = [ "i like dog", "i love coffee", "i hate milk"]

In [3]:
word_list = " ".join(sentences).split()
print(word_list)
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)

['i', 'like', 'dog', 'i', 'love', 'coffee', 'i', 'hate', 'milk']


In [4]:
word_dict

{'love': 0, 'hate': 1, 'dog': 2, 'coffee': 3, 'milk': 4, 'like': 5, 'i': 6}

In [5]:
number_dict

{0: 'love', 1: 'hate', 2: 'dog', 3: 'coffee', 4: 'milk', 5: 'like', 6: 'i'}

In [6]:
# TextRNN Parameter
n_step = 2 # number of cells(= number of Step)
n_hidden = 5 # number of hidden units in one cell

In [7]:
def make_batch(sentences):
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])

    return input_batch, target_batch

In [8]:
# Model
X = tf.placeholder(tf.float32, [None, n_step, n_class]) # [batch_size, n_step, n_class]
Y = tf.placeholder(tf.float32, [None, n_class])         # [batch_size, n_class]

In [9]:
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [10]:
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
# outputs : [batch_size, n_step, n_hidden]
outputs = tf.transpose(outputs, [1, 0, 2]) # [n_step, batch_size, n_hidden]
outputs = outputs[-1] # [batch_size, n_hidden]
model = tf.matmul(outputs, W) + b # model : [batch_size, n_class]

In [12]:

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

prediction = tf.cast(tf.argmax(model, 1), tf.int32)

In [13]:
# Training
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [14]:
input_batch, target_batch = make_batch(sentences)

In [15]:
input_batch

[array([[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.]]), array([[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.]]), array([[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.]])]

In [16]:
target_batch

[array([1., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 1., 0., 0.])]

In [17]:
for epoch in range(5000):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

Epoch: 1000 cost = 0.120100
Epoch: 2000 cost = 0.023382
Epoch: 3000 cost = 0.008809
Epoch: 4000 cost = 0.004136
Epoch: 5000 cost = 0.002146


In [18]:
input = [sen.split()[:2] for sen in sentences]

In [19]:
predict =  sess.run([prediction], feed_dict={X: input_batch})
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n] for n in predict[0]])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
